# Prediciting Post Reaction using Naive Bayes

## Import Libraries

### Main Libraries

In [1]:
import pandas as pd
import numpy as np
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Scikit Learn

In [2]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

## Load Dataset

In [3]:
df_train = pd.read_csv('C:/Users/cherryb/Desktop/Personal Projects/Datasets/Telus - Fintech/results/postSenti_train.tsv', index_col='Unnamed: 0', sep='\t')
df_test = pd.read_csv('C:/Users/cherryb/Desktop/Personal Projects/Datasets/Telus - Fintech/results/postSenti_test.tsv', index_col='Unnamed: 0', sep='\t') 

In [4]:
# Separate features and target for training dataset
X_train = df_train.drop(['sentiment_class'], axis=1)
y_train = df_train['sentiment_class']

In [5]:
# Separate features and target for testing dataset
X_test = df_test.drop(['sentiment_class'], axis=1)
y_test = df_test['sentiment_class']

## Naive Bayes Classification

### Feature selection using RandomForestClassifier--feature_importances

In [6]:
# Instantiate RandomForestClassifier
rf = RandomForestClassifier(random_state=35).fit(X_train, y_train)
rf_feat_imp = rf.feature_importances_
# Feature importance
feature_importance = pd.DataFrame(rf_feat_imp, index=X_train.columns, columns=["Feat_Imp"])

In [7]:
#Sort in descending order to get Most Important to Least Important
feature_importance.sort_values("Feat_Imp",ascending=False, inplace=True)
feature_importance.reset_index(inplace=True)
feature_importance

,index,Feat_Imp
0,Topic 0,0.167904
1,hour_day,0.139367
2,Topic 1,0.109699
3,Topic 2,0.069233
4,Topic 7,0.067534
5,Topic 9,0.063990
6,Topic 5,0.060787
7,Topic 6,0.057733
8,Topic 8,0.055409
9,Topic 4,0.051431


In [14]:
# Instantiate NB
clf = MultinomialNB()

# Function to get the accuracy of the model
def get_accuracy(xtrain, ytrain, xtest, ytest):
    clf.fit(xtrain**2, ytrain)
    ypred = clf.predict(xtest**2)
    return accuracy_score(ypred, ytest)*100

In [15]:
score_dict = {}
# Store accuracies in score_dict
for i in range(1, len(feature_importance)):
    features = feature_importance.iloc[:i]
    predictor = list(features['index'])
    key = 'top' + str(i)
    score_dict[key] = get_accuracy(X_train[predictor], y_train, X_test[predictor], y_test)

In [16]:
# Check scores to determine how many features will be used
score_dict

{'top1': 60.204081632653065,
 'top2': 60.204081632653065,
 'top3': 60.204081632653065,
 'top4': 57.14285714285714,
 'top5': 57.6530612244898,
 'top6': 52.04081632653062,
 'top7': 52.55102040816326,
 'top8': 48.97959183673469,
 'top9': 46.93877551020408,
 'top10': 45.40816326530612,
 'top11': 43.36734693877551,
 'top12': 41.3265306122449,
 'top13': 41.3265306122449,
 'top14': 41.83673469387755,
 'top15': 41.83673469387755,
 'top16': 42.3469387755102,
 'top17': 42.3469387755102}

In [17]:
# Top 3 Feature Importance
top_feature = feature_importance.iloc[:3]
predictor = list(top_feature['index'])
predictor

['Topic 0', 'hour_day', 'Topic 1']

### Hyperparameter Optimization using GridSearch

In [18]:
param_grid = {'alpha': [1, 0.5, 0.1, 0.05, 0.01, 0.005],
              'class_prior': [None],
              'fit_prior': [True]}

In [19]:
# Instantiate GridSearchCV
grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5)

In [20]:
# Fit X_train; Squared the train dataset because the model does not accept negative values
grid_search.fit(X_train[predictor]**2, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'alpha': [1, 0.5, 0.1, 0.05, 0.01, 0.005], 'class_prior': [None], 'fit_prior': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [21]:
print("Best estimator:")
print(grid_search.best_estimator_)
print("Best parameters:")
print(grid_search.best_params_)
print("Best score:")
print(grid_search.best_score_)

Best estimator:
MultinomialNB(alpha=1, class_prior=None, fit_prior=True)
Best parameters:
{'alpha': 1, 'class_prior': None, 'fit_prior': True}
Best score:
0.5646606914212549


### Build NB Model

In [22]:
# Instantiate NB
NB = MultinomialNB(alpha=1, class_prior=None, fit_prior=True)
# Fit the training dataset on the NB; Squared the test dataset because the model does not accept negative values
NB.fit(X_train[predictor]**2, y_train)
# Predict the labels on validation dataset
y_pred = NB.predict(X_test[predictor]**2)

### Evaluate NB Model

In [23]:
# Get Metrics
accuracy = accuracy_score(y_pred, y_test)*100
# Store metrics in dictionary
score_dict = {'NaiveBayes': pd.Series([accuracy], index=['AccuracyScore'])}
# Transform dictionary to df
df_metrics = pd.DataFrame(score_dict)
df_metrics

,NaiveBayes
AccuracyScore,60.204082


## Cross Validation

In [24]:
print('Mean Cross validated score -> ', np.mean(cross_val_score(NB, X_train[predictor]**2, y_train, cv=5)*100))

Mean Cross validated score ->  56.46681534064139
